# Group 04 | Data Import & Feature Engineering

-------------------------
Amber Curran (akc6be)

Manpreet Dhindsa (mkd8bb)

Quinton Mays (rub9ez)

---------------------------

## Load Data

To begin we create our Spark Session.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("group04DataImport") \
        .getOrCreate()

To reduce the possible errors and the increased runtime with having Spark infer the schema and to avoid hard-coding the column types we use the supplied data dictionary to create a Spark schema for our data. First, the data dictionary is read into a `pandas` dataframe on the driver node:

In [2]:
import pandas as pd

In [3]:
schemadf = pd.read_csv('/project/ds5559/fa21-group04/data/WiDS_Datathon_2020_Dictionary.csv')
schemadf = schemadf[schemadf['Variable Name'] != 'icu_admit_type'] # added to address error in data dictionary

Next, spark dataframe column type objects are mapped to the data types as they appear in the data dictionary:

In [4]:
from pyspark.sql.types import *
# define a python dictionary to map the data types from the data dictionary to the column types in spark schema.
data_types = {
    'integer': IntegerType(),
    'binary': IntegerType(),
    'numeric': FloatType(),
    'string': StringType()
}

A dictionary comprehension is then used to create a schema for the dataframe.

In [5]:
# define a pyspark schema for the dataframe
schema = StructType(
    [
        StructField(row['Variable Name'] , 
                    data_types[row['Data Type']], 
                    True) for index, row in schemadf.iterrows()
    ]
)

Next, the data is read in using the created schema:

In [6]:
# read in the data using the schema
df = spark.read.format('csv') \
    .schema(schema) \
    .option('header', True) \
    .load('/project/ds5559/fa21-group04/data/training_v2.csv')

After reviewing the data manually, it was identified that `bmi` was type Float, not String as the data dictionary indicated, so this change was manually coded.

In [7]:
df = df.withColumn("bmi",df.bmi.cast(FloatType()))

Next, our group made the decision to remove the columns that contained more than 50% missing data from the dataset. We believe that these columns contained so many missing rows that imputing them could affect the distribution of the data and that any feature engineering based on the missing features could be difficult due to our lack of experience with the subject matter.

*Note that the dataframe was selected from its own columns excluding the columns to drop instead of dropping the columns, as this coerced all columns to type string as a byproduct.*

In [8]:
import pyspark.sql.functions as F
null_counts = df.select([((F.count(F.when(F.isnan(c) | F.col(c).isNull() | F.col(c).contains('NA'), c)))*100/df.count()).alias(c)  for c in df.columns]).collect()[0].asDict()
to_drop = [k for k, v in null_counts.items() if v > 50]

In addition, based on the data dictionary, the columns labeled in Category "APACHE covariate" are all factors in the variables `apache_4a_icu_death_prob` and `apache_4a_hospital_death_prob` and therefore we can remove.

In [9]:
apachelist = ['albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'apache_post_operative', 'arf_apache', 
                'bilirubin_apache', 'bun_apache', 'creatinine_apache', 'fio2_apache', 'gcs_eyes_apache', 'gcs_motor_apache', 
                'gcs_unable_apache', 'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache', 'hematocrit_apache', 
                'intubated_apache', 'map_apache', 'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache', 'ph_apache', 
                'resprate_apache', 'sodium_apache', 'temp_apache', 'urineoutput_apache', 'ventilated_apache', 'wbc_apache']
to_drop = to_drop + apachelist

Our group also removed the columns labeled in Category indicating "noninvasive", as according to the data dictionary they are included in the equivalent measurement which are measured either invasive or non invasive.

In [10]:
noninvasivelist = ['d1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min', 
                'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min',
                'h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min', 'h1_sysbp_noninvasive_max', 'h1_sysbp_noninvasive_min']
to_drop = to_drop + noninvasivelist

Next, we examined if the columns `patient_id` and `encounter_id` provided any information, or simply serve as a row IDs:

In [11]:
df.select('patient_id').distinct().count()

91713

In [12]:
df.select('encounter_id').distinct().count()

91713

In [13]:
df.count()

91713

As the number of distinct `patient_id` and `encounter_id`'s is equal to the number of rows in the dataset, we drop these columns:

In [14]:
to_drop = to_drop + ['encounter_id', 'patient_id']

In [16]:
df = df.select([col for col in df.columns if col not in to_drop])

Next, a set of columns that contain the minimum and maximum values for certain levels drawn during the first hour or day in the hospital are converted into a single column containing the average of the two values and the original columns are removed from the dataset.

In [19]:
def range_vitals (df, maxvalcol, minvalcol):
    rangeval = df.withColumn(colName = 'avgmaxto'+minvalcol, col = (F.col(maxvalcol)+F.col(minvalcol))/2)
    newrange = rangeval.drop(maxvalcol, minvalcol)
    return newrange

In [20]:
avgcol = [('d1_diasbp_max', 'd1_diasbp_min'), ('d1_heartrate_max', 'd1_heartrate_min'), ('d1_mbp_max', 'd1_mbp_min'), 
            ('d1_resprate_max', 'd1_resprate_min'), ('d1_spo2_max', 'd1_spo2_min'), ('d1_sysbp_max', 'd1_sysbp_min'), 
            ('d1_temp_max', 'd1_temp_min'), ('h1_diasbp_max', 'h1_diasbp_min'), ('h1_heartrate_max', 'h1_heartrate_min'), 
            ('h1_mbp_max', 'h1_mbp_min'), ('h1_resprate_max', 'h1_resprate_min'), ('h1_spo2_max', 'h1_spo2_min'), 
            ('h1_sysbp_max', 'h1_sysbp_min'), ('h1_temp_max', 'h1_temp_min'), ('d1_bun_max', 'd1_bun_min'), ('d1_calcium_max', 
            'd1_calcium_min'), ('d1_creatinine_max', 'd1_creatinine_min'), ('d1_glucose_max', 'd1_glucose_min'), 
            ('d1_hco3_max', 'd1_hco3_min'), ('d1_hemaglobin_max', 'd1_hemaglobin_min'), ('d1_hematocrit_max', 'd1_hematocrit_min'), 
            ('d1_platelets_max', 'd1_platelets_min'), ('d1_potassium_max', 'd1_potassium_min'), ('d1_sodium_max', 'd1_sodium_min'), 
            ('d1_wbc_max', 'd1_wbc_min')]

for colnam in avgcol:
    avgdf = range_vitals(df, colnam[0], colnam[1])
    df = avgdf

In [24]:
#df.columns

Finally, all missing values in the string columns are converted to `None` type so that they can be later be imputed or dropped as necessary with ease.

In [22]:
df = df.replace(to_replace='NA',
                value=None)

In [25]:
df.write.parquet("/project/ds5559/fa21-group04/data/df.parquet")